## 简介
__预测用户在2016年7月领取优惠券后15天以内的使用情况__
- 输入：      

| Table1 | Table2 | Table3 | 
| --- | --- | --- | 
| 用户线下消费和优惠券领取行为： |  用户线上点击/消费和优惠券领取行为 | 用户O2O线下优惠券使用预测样本 |

- 输出： Table 4和sample_submission.csv
***
整理后的新数据集有6个特征（包括标签）
- 后续操作主要利用新数据__train_df_new__继续进行

- 当存在train_df_new.csv文件时，只运行__1简介,2导入,4去除空值__部分即可

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.tree import DecisionTreeClassifier                                  # 决策树
from sklearn.model_selection import train_test_split                             # 数据集划分
from datetime import datetime, timedelta                                         # 日期处理
from chinese_calendar import is_holiday, get_holidays                            # 中国节假日
import os                                                                        # 检查文件（模型文件）是否存在，防止覆盖
import pickle                                                                    # 导入导出模型

%matplotlib inline

## 导入训练数据、测试数据

In [103]:
%%time
train_org = pd.read_csv("../Data/ccf_offline_stage1_train.csv")                    # ndarray
train_df_org = pd.DataFrame(train_org)                                             # dataframe
train_df_org.head()

Wall time: 4.45 s


In [104]:
train_df = train_df_org.iloc[:,3:]
train_df.head()

,Discount_rate,Distance,Date_received,Date
0,NaN,0.0,NaN,20160217.0
1,150:20,1.0,20160528.0,NaN
2,20:1,0.0,20160217.0,NaN
3,20:1,0.0,20160319.0,NaN
4,20:1,0.0,20160613.0,NaN


In [105]:
test_org = pd.read_csv("../Data/ccf_offline_stage1_test_revised.csv")         # ndarray
test_df_org = pd.DataFrame(test_org)                                             # dataframe
test_df_org.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,20160712
1,6949378,1300,3429,30:5,NaN,20160706
2,2166529,7113,6928,200:20,5.0,20160727
3,2166529,7113,1808,100:10,5.0,20160727
4,6172162,7605,6500,30:1,2.0,20160708


In [106]:
test_df = test_df_org.iloc[:,3:]
test_df.head()

,Discount_rate,Distance,Date_received
0,30:5,1.0,20160712
1,30:5,NaN,20160706
2,200:20,5.0,20160727
3,100:10,5.0,20160727
4,30:1,2.0,20160708


### 数据类型

In [107]:
train_df['Discount_rate'].unique()

array([nan, '150:20', '20:1', '200:20', '30:5', '50:10', '10:5', '100:10',
       '200:30', '20:5', '30:10', '50:5', '150:10', '100:30', '200:50',
       '100:50', '300:30', '50:20', '0.9', '10:1', '30:1', '0.95',
       '100:5', '5:1', '100:20', '0.8', '50:1', '200:10', '300:20',
       '100:1', '150:30', '300:50', '20:10', '0.85', '0.6', '150:50',
       '0.75', '0.5', '200:5', '0.7', '30:20', '300:10', '0.2', '50:30',
       '200:100', '150:5'], dtype=object)

In [108]:
train_df['Distance'].unique()

array([ 0.,  1., nan,  2., 10.,  4.,  7.,  9.,  3.,  5.,  6.,  8.])

In [109]:
low_len = train_df.shape[0]
low_len

1754884

## 数据预处理

In [110]:
# 'label': Date与Date_received相差15天内
train_df_new = pd.DataFrame(columns=['discount', 'threshold', 'distance', 'has_holidays', 'is_holiday', 'label'])

### 整理discount -> 'discount', 'thresold'
- nan:nan(不处理)     nan                nan
- 'A:B'： 满A减B      int(B)/int(A)      A
- '0.n'： 折数        float('0.n')       0

In [111]:
def process_discount(discountSeries_i):
    s = discountSeries_i
    if type(s)!=str and np.isnan(s):
        return  float(1), 0
    if ':'in s:
        l = s.split(':')
        threshold, minus = int(l[0]), int(l[1])
        return (threshold-minus)/threshold, threshold
    elif '.'in s:
        return float(s), 0

- Wall time: 1min 24s

In [112]:
%%time
l_discount, l_threshold = [], []
for i in range(low_len):
    result = process_discount(train_df['Discount_rate'][i])
    l_discount.append(result[0])
    l_threshold.append(result[1])
print(l_discount[:5], l_threshold[:5])

[1.0, 0.8666666666666667, 0.95, 0.95, 0.95] [0, 150, 20, 20, 20]
Wall time: 4min 4s


In [113]:
train_df_new['discount'] = l_discount
train_df_new['threshold'] = l_threshold
train_df_new.head(5)

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,NaN,NaN,NaN,NaN
1,0.866667,150,NaN,NaN,NaN,NaN
2,0.950000,20,NaN,NaN,NaN,NaN
3,0.950000,20,NaN,NaN,NaN,NaN
4,0.950000,20,NaN,NaN,NaN,NaN


### 整理distance

In [114]:
train_df_new['distance'] = train_df['Distance']
train_df_new.head()

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,0.0,NaN,NaN,NaN
1,0.866667,150,1.0,NaN,NaN,NaN
2,0.950000,20,0.0,NaN,NaN,NaN
3,0.950000,20,0.0,NaN,NaN,NaN
4,0.950000,20,0.0,NaN,NaN,NaN


### 整理Date_received
- 当天是否是节假日或周末
- 当天往后15天，是否有节假日
***
标签属性在于Date_received与Date的天数差是否大于15天

In [115]:
def process_date(dateSeries_i):
    s = dateSeries_i
    if np.isnan(s):
        return  False, False
    else:
        d = datetime.strptime(str(int(s)), '%Y%m%d')                                     # date
        d_e = d + timedelta(days = 15)
        return get_holidays(d, d_e)!=[], is_holiday(d)

- Wall time: 11min 26s

In [116]:
%%time
# 整理Date_received
l_has, l_is = [], []
for i in range(low_len):
    result = process_date(train_df['Date_received'][i])
    l_has.append(result[0])
    l_is.append(result[1])
print(l_has[:5], l_is[:5])

[False, True, True, True, True] [False, True, False, True, False]
Wall time: 19min 49s


In [117]:
train_df_new['has_holidays'] = l_has
train_df_new['is_holiday'] = l_is
train_df_new.head()

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,0.0,False,False,NaN
1,0.866667,150,1.0,True,True,NaN
2,0.950000,20,0.0,True,False,NaN
3,0.950000,20,0.0,True,True,NaN
4,0.950000,20,0.0,True,False,NaN


### 整理Date（label）

In [118]:
def process_label(received_i, date_i):
    s, e = received_i, date_i
    if np.isnan(s) or np.isnan(date_i):
        return  False
    else:
        d_s = datetime.strptime(str(int(s)), '%Y%m%d')
        d_e = datetime.strptime(str(int(e)), '%Y%m%d')
        minus = (d_e - d_s).days
        return minus<15

- Wall time: 2min 47s

In [119]:
%%time
# 整理Date
l_isbuy = []
for i in range(low_len):
    result = process_label(train_df['Date_received'][i], train_df['Date'][i])
    l_isbuy.append(result)
print(l_isbuy[:5])

[False, False, False, False, False]
Wall time: 2min 59s


In [120]:
train_df_new['label'] = l_isbuy
train_df_new[train_df_new['label'].notnull()].head()

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,0.0,False,False,False
1,0.866667,150,1.0,True,True,False
2,0.950000,20,0.0,True,False,False
3,0.950000,20,0.0,True,True,False
4,0.950000,20,0.0,True,False,False


In [122]:
### 保存train_df_new
train_df_new.to_csv('./train_df_new.csv',sep=',',index=False) 
train_df_new.head()

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,0.0,False,False,False
1,0.866667,150,1.0,True,True,False
2,0.950000,20,0.0,True,False,False
3,0.950000,20,0.0,True,True,False
4,0.950000,20,0.0,True,False,False


## 去除空值（决策树）

1754884个数据，空值率分别为：

| discount | threshold | distance | has_holidays | is_holiday | label |
| --- | --- | --- | --- |  --- | --- | 
| 0.39979964487681235 | 0.39979964487681235. | 0.060404562352839274 | 0.39979964487681235 |  0.39979964487681235 | 0.9570444542203359 |  

- ["discount", "threshold", "distance", "has_holidays", "is_holiday", "label"]

In [127]:
### 导入train_df_new
train_df_new = pd.read_csv('./train_df_new.csv')
train_df_new.head()

,discount,threshold,distance,has_holidays,is_holiday,label
0,1.000000,0,0.0,False,False,False
1,0.866667,150,1.0,True,True,False
2,0.950000,20,0.0,True,False,False
3,0.950000,20,0.0,True,True,False
4,0.950000,20,0.0,True,False,False


In [128]:
def count_nan_rate(series):
    return series.isnull().sum()/low_len

In [129]:
# 统计空值数量
train_df_new['discount'].isnull().sum(), train_df_new['threshold'].isnull().sum(), \
train_df_new['distance'].isnull().sum(), train_df_new['has_holidays'].isnull().sum(), \
train_df_new['is_holiday'].isnull().sum(), train_df_new['label'].isnull().sum(), \

(0, 0, 106003, 0, 0, 0)

In [12]:
# 统计空值比例
count_nan_rate(train_df_new['discount']), count_nan_rate(train_df_new['threshold']),    \
count_nan_rate(train_df_new['distance']), count_nan_rate(train_df_new['has_holidays']), \
count_nan_rate(train_df_new['is_holiday']), count_nan_rate(train_df_new['label'])

(0.39979964487681235,
 0.39979964487681235,
 0.060404562352839274,
 0.39979964487681235,
 0.39979964487681235,
 0.9570444542203359)

### 自定义函数

In [93]:
# 训练某个缺少值并得到训练模型,(not_null_df是全局变量)
def dt_fit_train(predictName):
    # 根据predictName整理数据集
    if predictName=='distance':
        exe_l = ['discount','threshold', 'has_holidays', 'is_holiday', 'label', 'distance']
        X, Y = not_null_df[exe_l[:5]], not_null_df[exe_l[5]]
    elif predictName in ['discount','threshold']:
        exe_l = ['has_holidays', 'is_holiday', 'distance', 'discount','threshold']                    # discount列不存在其他四列非空
        X, Y = not_null_df[exe_l[:3]], not_null_df[exe_l[3:]]
    elif predictName in ['has_holidays', 'is_holiday']:
        exe_l = ['label', 'distance', 'discount','threshold', 'has_holidays', 'is_holiday']
        X, Y = not_null_df[exe_l[:4]], not_null_df[exe_l[4:]]
    elif predictName=='label':
        exe_l = ['discount','threshold', 'has_holidays', 'is_holiday', 'distance', 'label']
        X, Y = not_null_df[exe_l[:5]], not_null_df[exe_l[5]]
        
    # 如果Y有两列，拆分分别训练
    print(X.shape,Y.shape)
    if len(Y.shape)>1:
        Y = Y[predictName]
#     print(X.head(),Y.head())
    
    # 分割训练集
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
    print(X_train.head(),Y_train.head())
    
    # 训练
    predict_clr = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    predict_clr.fit(X_train, Y_train.astype('int'))
    
    
    # 测试效果
    score = predict_clr.score(X_test, Y_test.astype('int'))     
    
    return predict_clr, score

In [94]:
# 导出indexs文件，存储特定查询的indexs
def exe_indexs(predictName):
    if predictName=='distance':
        indexs_0 = train_df_new[train_df_new['discount'].notnull()&train_df_new['threshold'].notnull()&train_df_new['has_holidays'].notnull()\
&train_df_new['is_holiday'].notnull()&train_df_new['label'].notnull()&train_df_new['distance'].isnull()].index
    elif predictName in ['discount','threshold']:
        indexs_0 = train_df_new[train_df_new['distance'].notnull()&train_df_new['has_holidays'].notnull()&train_df_new['is_holiday'].notnull()\
&train_df_new['discount'].isnull()].index                                                   # discount列不存在其他四列非空，不计算label
    elif predictName in ['has_holidays', 'is_holiday']:
        indexs_0 = train_df_new[train_df_new['discount'].notnull()&train_df_new['threshold'].notnull()&train_df_new['distance'].notnull()\
&train_df_new['label'].notnull()&train_df_new['has_holidays'].isnull()].index
    elif predictName=='label':
        indexs_0 = train_df_new[train_df_new['discount'].notnull()&train_df_new['threshold'].notnull()&train_df_new['has_holidays'].notnull()\
&train_df_new['is_holiday'].notnull()&train_df_new['distance'].notnull()&train_df_new['label'].isnull()].index
    
    print(indexs_0[:20])
    indexs = pd.DataFrame(list(indexs_0))
    print(indexs.head(20))
    indexs.to_csv('./indexs_%s.csv'%predictName, sep=',', index=False)

In [95]:
# 利用模型预测缺失值，直接在train_df_new里修改
def dt_pre(predictName, dtClassifier):
    indexs = list(pd.read_csv('./indexs_%s.csv'%predictName).iloc[:,0])
    
    # 分割数据集df
    columns = ["discount","threshold","distance","has_holidays","is_holiday","label"]
    columns_sep = [s for s in columns if s!=predictName]
    X_test = train_df_new.reindex(indexs)[columns_sep]
#     print('X_test', X_test.head())

    # 预测
    Y_pred = list(dtClassifier.predict(X_test))
    print(Counter(Y_pred))
    
    # 更改数据
    train_df_new.loc[indexs, predictName]  = Y_pred

    print("数据成功更改")
#     print(train_df_new[predictName])

### 预测distance
- 67165个数据，占比约4%

In [47]:
# 不为空的子集，作为训练集
not_null_df = train_df_new[train_df_new['discount'].notnull() & train_df_new['threshold'].notnull() \
                          & train_df_new['distance'].notnull() & train_df_new['has_holidays'].notnull()                \
                          & train_df_new['is_holiday'].notnull()  & train_df_new['label'].notnull()]

print(len(not_null_df))
not_null_df.head()

75382


,discount,threshold,distance,has_holidays,is_holiday,label
6,0.950000,20.0,0.0,True,False,False
33,0.950000,20.0,0.0,True,True,True
38,0.750000,20.0,0.0,True,False,True
69,0.833333,30.0,0.0,True,False,True
75,0.833333,30.0,0.0,True,False,False


In [32]:
predictName = 'distance'
# 训练
model, score = dt_fit_train(predictName)
best_score = 0.7002977667493796

print('正确率： ',score)

(67165,)
discount        0
threshold       0
has_holidays    0
is_holiday      0
label           0
dtype: int64
0
正确率：  0.7002977667493796


In [33]:
# 导出模型
mes = datetime.now().strftime('%y%m%d_%H')                            # 以时间为导出模型文件名
## 该模型信息是否存在
if not os.path.isfile('../Model/model_dis_%s.pkl'%mes): 
    if score>best_score:
        with open('../Model/model_dis_%s.pkl'%mes, 'wb') as f:
            pickle.dump(model, f)                                           # 导出模型
            print("模型已导出")
            
# 导入模型
# with open('../Model/model%s.pkl'%mes, 'rb') as f:
#     model = pickle.load(f)                                          
#     print("模型已导入")

In [34]:
exe_indexs(predictName)

In [35]:
dt_pre(predictName, model)
train_df_new[predictName].isnull().sum(), '增加了：', 106003-train_df_new[predictName].isnull().sum()

Counter({0: 8152, 10: 60, 3: 3, 4: 2})
数据成功更改


(97786, '增加了：', 8217)

### 预测discount派生得到的两列'discount', 'threshold'
- 75382个数据

In [96]:
# 不为空的子集，作为训练集
not_null_df = train_df_new[train_df_new['discount'].notnull() & train_df_new['threshold'].notnull() \
                          & train_df_new['distance'].notnull() & train_df_new['has_holidays'].notnull()                \
                          & train_df_new['is_holiday'].notnull()  & train_df_new['label'].notnull()]

print(len(not_null_df))
not_null_df.head()

75382


,discount,threshold,distance,has_holidays,is_holiday,label
6,0.950000,20.0,0.0,True,False,False
33,0.950000,20.0,0.0,True,True,True
38,0.750000,20.0,0.0,True,False,True
69,0.833333,30.0,0.0,True,False,True
75,0.833333,30.0,0.0,True,False,False


In [97]:
predictName = 'discount'
# 训练
model, score = dt_fit_train(predictName)
best_score = 1.0

print('正确率： ',score)

(75382, 3) (75382, 2)
        has_holidays is_holiday  distance
152428          True      False       0.0
12717           True      False       1.0
32714           True       True       0.0
1743075         True      False       2.0
818449          True       True       1.0 152428     0.833333
12717      0.833333
32714      0.900000
1743075    0.833333
818449     0.900000
Name: discount, dtype: float64
正确率：  1.0


In [98]:
# 导出模型
mes = datetime.now().strftime('%y%m%d_%H')                            # 以时间为导出模型文件名
## 该模型信息是否存在
if not os.path.isfile('../Model/model_count_%s.pkl'%mes): 
    if score>best_score:
        with open('../Model/model_count_%s.pkl'%mes, 'wb') as f:
            pickle.dump(model, f)                                           # 导出模型
            print("模型已导出")
            
# 导入模型
# with open('../Model/model%s.pkl'%mes, 'rb') as f:
#     model = pickle.load(f)                                          
#     print("模型已导入")

In [101]:
train_df_new[train_df_new['has_holidays'].notnull()&train_df_new['discount'].isnull()]

,discount,threshold,distance,has_holidays,is_holiday,label


In [99]:
exe_indexs(predictName)

predictName in ['discount','threshold']
Int64Index([], dtype='int64')
Empty DataFrame
Columns: []
Index: []


In [40]:
dt_pre(predictName, model)
train_df_new[predictName].isnull().sum(), '增加了：', 701602-train_df_new[predictName].isnull().sum()

EmptyDataError: No columns to parse from file

In [ ]:
predictName = 'threshold'
# 训练
model, score = dt_fit_train(predictName)
best_score = 0.7002977667493796

print('正确率： ',score)

In [ ]:
# 导出模型
mes = datetime.now().strftime('%y%m%d_%H')                            # 以时间为导出模型文件名
## 该模型信息是否存在
if not os.path.isfile('../Model/model_thre_%s.pkl'%mes): 
    if score>=best_score:
        with open('../Model/model_thre_%s.pkl'%mes, 'wb') as f:
            pickle.dump(model, f)                                           # 导出模型
            print("模型已导出")
            
# 导入模型
# with open('../Model/model%s.pkl'%mes, 'rb') as f:
#     model = pickle.load(f)                                          
#     print("模型已导入")

In [ ]:
dt_pre(predictName, model)
train_df_new[predictName].isnull().sum(), '增加了：', 701602-train_df_new[predictName].isnull().sum()

### 预测date_received派生得到'has_holidays', 'is_holiday'

### 预测label（date）

### 空值数量

## 决策树训练

### AUC

### 导出模型

### 可视化